In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [4]:
# 検索ツールの定義
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]

In [5]:
tool.invoke("Langgraphのノードとは？")

[{'url': 'https://www.creationline.com/tech-blog/author/higuchi/75504',
  'content': '## 「ノード」と「エッジ」\n\nまず「グラフ」という概念において、点である「ノード」と線である「エッジ」という構成要素を押さえておく必要がありそうです。LangGraphにおいても、処理を担当する「ノード」と処理同士を接続する「エッジ」という、次のような図になりそうです。\n\n## ２つのノードを接続する\n\n `value` を `1` にする `node1`\n `value` を `2` にする `node2`\n\nと２つのノードがあり、\n\n 処理は `node1` から `node2` に進む\n\nというエッジで２つのノードを接続するとします。また、\n\n `node1` から始まる\n `node2` で終わる\n\nという流れとします。ここで `value` の初期値を `0` として開始するLangGraphスクリプトを書いてみました。'},
 {'url': 'https://zenn.dev/pharmax/articles/8796b892eed183',
  'content': '```\n\nreducerを使用することで、状態の更新時に特定のロジックを介することが可能になります。\n\n## Node\n\nNodeは、Graph上で実際のステップを実行するコンポーネントです。例えば実際にLangChainを使用してLLMのモデルを呼び出す処理は、Node上で行います。\n\nNodeでは、`(State、Config) -> 変更するState`というシグネチャを持つ関数を使うことができます。\n\n```\nfrom typing import Annotated\nfrom typing_extensions import TypedDict\nfrom langchain_core.runnables import RunnableConfig\nfrom langgraph.graph import StateGraph\n\n# Stateを宣言\nclass State(TypedDict):\n    value: str\n\n# N

In [8]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# Stateクラスの定義
class State(TypedDict):
    messages: Annotated[list, add_messages]

# グラフのインスタンスを作成
graph_builder = StateGraph(State)

# 言語モデルの定義
llm = ChatOpenAI(model_name=MODEL_NAME)

# 変更点：ツール定義の紐づけ
llm_with_tools = llm.bind_tools(tools)

# チャットボットノードの作成
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# グラフにチャットボットノードを追加
graph_builder.add_node("chatbot", chatbot)

In [9]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ツールノードの作成
tool_node = ToolNode(tools)

# グラフにツールノードを追加
graph_builder.add_node("tools", tool_node)

# 条件付エッジの作成
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
)

# ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
# ツールからチャットボットへの戻りエッジを作成
graph_builder.add_edge("tools", "chatbot")

# 開始ノードの指定
graph_builder.set_entry_point("chatbot")

# 記憶を持つ実行可能なステートグラフの作成
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [10]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
1足す2は？
1足す2は3です。何か他に知りたいことはありますか？
1メートル以上の魚は？

[{"url": "https://www.kuromon.jp/blog/industry/1018/", "content": "鮮魚川崎について\n 川崎ブログ\n ご注文方法\n\n HOME\n 鮮魚川崎について\n 川崎ブログ\n ご注文方法\n お問い合わせ\n プライバシーポリシー\n\n HOME >\n 業界情報 >\n 1m超える新種の深海魚「ヨコヅナイワシ」発見 静岡沖 駿河湾\n\n業界情報\n\n## 1m超える新種の深海魚「ヨコヅナイワシ」発見 静岡沖 駿河湾\n\n静岡県沖の駿河湾で、体長1メートルを超える大型の新種の深海魚が見つかり、「ヨコヅナイワシ」と名付けられました。発見したグループは「これほど大型の魚が見つからずにいたのは珍しい」としています。\n\n海洋研究開発機構の研究グループは、2016年に静岡県沖の駿河湾の深海で生物の調査を行ったところ、水深2000メートル余りでこれまで知られていない大型の深海魚を4匹捕獲しました。\n\n4匹は、いずれも体長が1メートルを超えていて、最も大きいものは1メートル30センチ余り、重さはおよそ25キロあったということです。\n\n形態や遺伝子の解析から、「セキトリイワシ」という深海魚の仲間の新種で、この仲間の中では最も大きいことから、研究グループは「ヨコヅナイワシ」と名付けました。\n\n「ヨコヅナイワシ」のうろこは鮮やかな青色で、胃の内容物や食物連鎖の中での位置を調べる最新の分析方法から、駿河湾の深海の食物連鎖の最上位に位置することが分かったということです。\n\nまた、研究グループは、深海に設置したカメラで、1メートルを超える「ヨコヅナイワシ」が、海底付近をゆっくりと泳ぐ姿の撮影にも成功しています。\n\n海洋研究開発機構の藤原義弘上席研究員は「大型の魚が見つからずにいたのは珍しい。深海には私たちの知らない生物の多様性があり、深海での漁業も増えているので、生態系を崩さないよう注意が必要だ」と話していました。\n\n## 「ヨコヅナイワシ」の特徴 [...] 最近の魚達\n\n### 初サンマ、キロ12万円の最高値　不漁で入荷わず